In [1]:
import pandas as pd
import numpy as np
from tools import seq_to_num, acc_score, prep_submit

In [2]:
df_train = pd.read_csv('../data/train.csv', index_col=0)
df_test = pd.read_csv('../data/test.csv', index_col=0)

In [3]:
!pip install tensorflow

twisted 18.7.0 requires PyHamcrest>=1.9.0, which is not installed.
distributed 1.21.8 requires msgpack, which is not installed.
You are using pip version 10.0.1, however version 18.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [4]:
train_X, train_y = seq_to_num(df_train.Sequence, pad=False)

Using TensorFlow backend.


In [19]:
def diff(data, only_seq=False, verbose=False):
    """
    Calculate next term if first difference is constant
    
    @returns:
        * list of sequences that have constant difference
        * list of the corresponding indices
        * list of predicted terms
    """
    sequences = []
    indices = []
    predictions = []
    for seq, idx in zip(data, data.index):
        if len(seq) < 3:
#             if verbose:
#                 print("Sequence is too small to calculate terms differences:", seq)
            continue
        last_elems = [seq[-1]]  # last elements of the corresponding differences
        diffs = [_ for _ in seq]
        for i in range(1, len(seq) - 1):
            diffs = [next - cur for cur, next in zip(diffs, diffs[1:])]
            last_elems.append(diffs[-1])
            uniques = np.unique(diffs)
            if len(uniques) == 1:
                if verbose:
                    print(f"Seq {seq[:5]}... has constant {i}-th difference {uniques[-1]}")
                sequences.append(seq)
                indices.append(idx)
                predictions.append(sum(last_elems))
                break
    if only_seq:
        return sequences
    else:
        return sequences, indices, predictions

In [20]:
test = [2, 4, 7, 11, 16]
diff(pd.Series([test], dtype='object'))

([[2, 4, 7, 11, 16]], [0], [22])

In [47]:
seq, ind, pred = diff(train_X, verbose=False)

In [48]:
len(seq)

2077

In [49]:
acc_score(train_y[ind], pred)  # first-order difference, perfect prediction

0.9046701974000962